In [1]:
!pip install mediapipe


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\USER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# Push-Up Counter & Plank Timer using Device Camera (OpenCV + Mediapipe)

import cv2
import mediapipe as mp
import numpy as np
import time

# Initialize mediapipe pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_draw = mp.solutions.drawing_utils

# Initialize counters
pushup_count = 0
stage = None
plank_start_time = None
plank_holding = False

# Angle calculation helper
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Open webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    landmarks = results.pose_landmarks

    if landmarks:
        mp_draw.draw_landmarks(frame, landmarks, mp_pose.POSE_CONNECTIONS)

        h, w, _ = frame.shape
        lm = landmarks.landmark

        # Get coordinates for push-up
        shoulder = [lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * w,
                    lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * h]
        elbow = [lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].x * w,
                 lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].y * h]
        wrist = [lm[mp_pose.PoseLandmark.LEFT_WRIST.value].x * w,
                 lm[mp_pose.PoseLandmark.LEFT_WRIST.value].y * h]

        angle = calculate_angle(shoulder, elbow, wrist)

        # Push-up counter logic
        if angle < 70:
            stage = "down"
        if angle > 160 and stage == "down":
            stage = "up"
            pushup_count += 1

        # Get coordinates for plank
        shoulder_y = lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * h
        hip_y = lm[mp_pose.PoseLandmark.LEFT_HIP.value].y * h
        ankle_y = lm[mp_pose.PoseLandmark.LEFT_ANKLE.value].y * h

        if abs(shoulder_y - hip_y) < 40 and abs(hip_y - ankle_y) < 40:
            if not plank_holding:
                plank_start_time = time.time()
                plank_holding = True
        else:
            plank_holding = False

    # Display push-up count
    cv2.rectangle(frame, (0,0), (250,80), (245,117,16), -1)
    cv2.putText(frame, f'Push-Ups: {pushup_count}', (10,40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

    # Display plank timer
    if plank_holding:
        plank_time = int(time.time() - plank_start_time)
        cv2.putText(frame, f'Plank: {plank_time}s', (10,70),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
    else:
        cv2.putText(frame, f'Plank: 0s', (10,70),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

    cv2.imshow('Push-Up & Plank Tracker', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
